In [ ]:
%runfile zeta-types-smc/zetatypes/util/lazylist.py
%runfile zeta-types-smc/zetatypes/ts/monoid.py
%runfile zeta-types-smc/zetatypes/ts/ring.py
%runfile zeta-types-smc/zetatypes/util/berlekamp.py
%runfile zeta-types-smc/zetatypes/ts/complex.py

In [ ]:
%runfile zeta-types-smc/zetatypes/mf/mf.py
%runfile zeta-types-smc/zetatypes/mf/library.py

In [ ]:
defineTS()

In [ ]:
TSC = ComplexTannakianSymbols()

In [ ]:
def approx_sum(terms, delta = 0.0001, taillen = 10):
    
    #G = Graphics()
    
    tails = [0] * taillen
    
    i = 0
    
    if hasattr(terms, 'offset'):
        i = terms.offset
    
    while i <= taillen or sum(abs(tails[0] - x) for x in tails) > delta: #abs(tails[0] - tails[-1]) + abs(tails[0] - tails[1]) > delta:
        tails = [tails[0] + terms[i]] + tails[:-1]
                
        #print(abs(tails[0] - tails[-1]))
        #G += point((real(tails[0]), imag(tails[0])))
        i += 1
        #if i % 1000 == 0:
            #print("Progress report, delta is now:")
            #print(sum(abs(tails[0] - x) for x in tails))
            #print(abs(tails[0] - tails[-1]))
    
    #print "Amount of terms: ", i
    #print tails[0]
    #G.show()
    return tails[0]

In [ ]:
def m_approx(f, z):
    return approx_sum(LazyList(lambda n: f(n) * z^(n - 1), cache = False, offset = 1))

def m_der_approx(f, z):
    return approx_sum(LazyList(lambda n: f(n) * (n - 1) * z^(n - 2), cache = False, offset = 2))

def m_der_n_approx(f, i, z):
    def prod(n, j):
        return n * prod(n - 1, j - 1) if j > 0 else 1

    return approx_sum(LazyList(lambda n: f(n) * prod(n - 1, i) * z^(n - 1 - i), cache = False, offset = i + 1))


#def smart_m_approx(f, z):
    #def rie(z):
        #return 1/(1 - z)
    #C = LazyList(lambda n: sum(moebius(n/d)*f(d) for d in divisors(n)), offset = 1)
    #print(C)
    #return approx_sum(LazyList(lambda i: - C[i + 1] * z^(i + 1) * m_approx(lambda n: 1, 1/(z^(i + 1)))))
    #return approx_sum(LazyList(lambda n: C[n] * rie(z^n), offset = 1))


In [ ]:
def amt_terms(r, bound = 1, delta = 0.001):
    if bound == 1:
        return ceil((log(delta) + log(1 - r))/log(r)) + 1
    raise ValueError("No known bound for error when a_n is bounded by " + bound)
    
def bounded_m_approx(f, z, bound = 1, delta = 0.001):
    r = abs(z)
    if r >= 1:
        raise ValueError("This sequence diverges!")
    else:
        N = amt_terms(r, bound = bound, delta = delta)
        return sum(f(n) * z^(n - 1) for n in range(1, N + 1))

In [ ]:
bounded_m_approx(moebius, 0.999)

In [ ]:
amt_terms(0.95)

In [ ]:
mob = LazyList(lambda n: moebius(n), offset = 1)
# TODO: give lazylist call function

m_mu = lambda z: bounded_m_approx(lambda n: mob[n], z)
#m_mu_der = lambda z: m_der_approx(moebius, z)
#m_mu_der_n = lambda n: lambda z: m_der_n_approx(moebius, n, z)

In [ ]:
def make_r_plot(r):
    #print "Current r: " + str(r)
    return plot(lambda x: arg(m_mu(CC(r * e^(I*x)))), (0, 2*pi), plot_points = 100)

In [ ]:
import time

start = time.time()
make_r_plot(.5).show()
end = time.time()
print(end - start)

In [ ]:
#plot(lambda x: m_mu(x) - (1/(x^2 - 1) + 2 - x) if abs(x) <= .9 else None, (-1, 1))

In [ ]:
#plot(lambda x: m_mu(x) if abs(x) <= .99 else None, (-1, 1))

In [ ]:
#plot(lambda x: m_mu_der(x) if abs(x) <= .9 else None, (-1, 1))

In [ ]:
plot(lambda x: (1 - x^2) * m_mu(x) if abs(x) <= .99 else None, (-1, 1))

In [ ]:
plot(lambda x: (-2*x)*m_mu(x) + (1 - x^2)*m_mu_der(x) if abs(x) <= .99 else None, (-1, 1))

In [ ]:
#complex_plot(lambda z: m_mu(z) if abs(z) <= .99 else NaN, (-1, 1), (-1, 1))

In [ ]:
plot3d(lambda x, y: abs(m_approx(lambda n: mob[n], CC(x + I*y))) if x*x + y*y <= (.9)^2 else -1, (-1, 1), (-1, 1))

In [ ]:
polar_plot(lambda x: abs(m_mu(CC(.9 * e^(I*x)))), (0, 2*pi))

In [ ]:
p = make_plot(0.97)

In [ ]:
p + sum(line([((2 * pi * i) / pr, 0), ((2 * pi * i) / pr, 5)], color="red") for pr in [11] for i in range(1, pr) if pr in Primes())

In [ ]:
def make_plot(r):
    print "Current r: " + str(r)
    return plot(lambda x: arg(m_mu(CC(r * e^(I*x)))), (0, 2*pi), plot_points=100)

plots = [make_plot(r) for r in sxrange(0, 1,.01)]
print "Finished!"

In [ ]:
anim = animate(plots)

In [ ]:
anim.save(filename = "gif.mp4", use_ffmpeg=True)

In [ ]:
anim.show()

In [ ]:
plots[-1]

In [ ]:
#plots[-2] + sum(line([((2 * pi * i) / pr, 0), ((2 * pi * i) / pr, 20)], color="red") for pr in [17, 19] for i in range(1, pr) if pr in Primes())

In [ ]:
for p in plots:
    p.axes_range(ymin=0, ymax=22)

In [ ]:
animate(plots).save(filename = "gif.mp4", use_ffmpeg=True)

In [ ]:
plot3d(lambda x, y: abs(m_mu(CC(x * e^(I*y)))), (0, .99), (0, 2*pi), plot_points = 200)